# Importing necessary libraries

In [1]:
!pip install nltk
!pip install numpy
!pip install pandas
!pip install requests
!pip install re
!pip install spacy
import nltk
from nltk.corpus import stopwords
from nltk.corpus import  state_union
from nltk.tokenize import PunktSentenceTokenizer
import requests
import pandas as pd
import numpy as np
import re
import json
import string
import spacy
from spacy.lang.en.examples import sentences
from spacy import displacy
import itertools
from nltk.probability import FreqDist

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


# First I have crawled data from movie reviews of theGuardian and have retained information of headline, the article, and the rating it received from the reviewer.
In this step we load data from a json file

In [3]:
f = open('data.json')
data = json.load(f)
train = pd.DataFrame(data)
print(train.head(10))

                                       news_headline  \
0  Godzilla x Kong: The New Empire review – breez...   
1  Mary Poppins review – Disney’s entertainment s...   
2  Kung Fu Panda 4 review – Jack Black and Awkwaf...   
3  Ryuichi Sakamoto: Opus review – a stark, emoti...   
4  The Persian Version review – feelgood Iranian-...   
5  Robot Dreams review – bittersweet buddy movie ...   
6  Late Night With the Devil review – diabolicall...   
7  Swatantra Veer Savarkar review – biopic of Hin...   
8  Irish Wish review – no cliche is left unturned...   
9  Drive-Away Dolls review – Ethan Coen’s lesbian...   

                                      news_firstline  rating  \
0  There’s a likable, light-hearted zip to the mo...       3   
1  Manic, magic, madcap … Julie Andrews is superb...       5   
2  The lead pair make a brilliant double act, but...       2   
3  In his last weeks of life, the Oscar-winning c...       4   
4  A perceptive and candid look at mother-daughte...       3   

# As we can see here there are 1196 reviews crawled from the website. Below you can see that tokenized sentences are made up of almost 330000 words

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   news_headline   1196 non-null   object
 1   news_firstline  1196 non-null   object
 2   rating          1196 non-null   int64 
 3   news_article    1196 non-null   object
dtypes: int64(1), object(3)
memory usage: 37.5+ KB


In [5]:
train_news = train["news_article"]

# First of all we will remove all of punctuations

In [6]:
def remove_punctuation(text):
    punct_list = list(string.punctuation)
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip()

# In this stage we will lowercase all of the words. then we would remove all of 's in the sentence because it doesn't contain any valuable information. Finally we would use word tokenizer of nltk to tokenize all of the sentences

In [7]:
tokenized_reviews=[]
# nltk.download('stopwords')
stop=stopwords.words()
for i, review in enumerate(train_news):
  temp = remove_punctuation(review.lower().replace("’s", '').strip())
  tokenized_reviews.append(nltk.word_tokenize(temp))


In [8]:
np.array(tokenized_reviews)

C:\Users\smrh1\AppData\Local\Temp\ipykernel_35196\1727641997.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(tokenized_reviews)


array([list(['it', 'was', 'a', 'strange', 'old', 'time', 'when', 'the', 'creature', 'feature', 'mash', 'up', 'godzilla', 'vs', 'kong', 'was', 'released', 'the', 'first', 'major', 'blockbuster', 'in', 'cinemas', 'since', 'covid', 'shuttered', 'them', 'all', 'a', 'year', 'prior', 'expectations', 'were', 'low', 'thanks', 'to', 'how', 'rotten', 'the', 'last', 'two', 'godzilla', 'films', 'had', 'been', 'but', 'thirst', 'for', 'something', 'anything', 'truly', 'escapist', 'was', 'high', 'and', 'the', 'big', 'screen', 'equivalent', 'of', 'a', 'kid', 'smashing', 'his', 'toys', 'together', 'became', 'an', 'unlikely', 'saviour', 'both', 'commercially', 'and', 'critically', 'three', 'years', 'later', 'with', 'normality', 'resuming', 'there', 'arguably', 'less', 'audience', 'demand', 'for', 'another', 'instalment', 'although', 'the', 'industry', 'could', 'definitely', 'do', 'with', 'another', 'monster', 'hit', 'the', 'strikes', 'leaving', 'the', 'first', 'few', 'months', 'of', '2024', 'a', 'little

In [9]:
stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]

# After tokeinzation we would remove all of the Stopwords because these words doesn't contain any valuable information.

In [10]:
for i, review in enumerate(tokenized_reviews):
 tokenized_reviews[i] = [word for word in review if word not in stopwords]

In [11]:
pd.DataFrame(tokenized_reviews)

,0,1,2,3,4,5,6,7,8,9,...,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202
0,strange,old,time,creature,feature,mash,godzilla,vs,kong,released,...,None,None,None,None,None,None,None,None,None,None
1,brilliant,entrancing,exhausting,thermonuclear,showtunes,richard,robert,sherman,disney,hybrid,...,None,None,None,None,None,None,None,None,None,None
2,cuddly,kung,fu,master,back,jack,black,returns,dumpling,loving,...,None,None,None,None,None,None,None,None,None,None
3,short,presenting,nothing,music,blank,screen,documentary,late,japanese,composer,...,None,None,None,None,None,None,None,None,None,None
4,daughter,family,nine,iranian,american,siblings,leila,layla,mohammadi,relationship,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,energetic,emotionally,reflective,movie,follows,rise,cuban,dancer,carlos,acosta,...,None,None,None,None,None,None,None,None,None,None
1192,animation,studio,laika,artisanal,cheese,maker,kids,’,movies,–,...,None,None,None,None,None,None,None,None,None,None
1193,good,performance,tom,hollander,’,save,stodgy,ungainly,strangely,reactionary,...,None,None,None,None,None,None,None,None,None,None
1194,nervous,translation,filipino,artist,film,maker,shireen,seno,former,stills,...,None,None,None,None,None,None,None,None,None,None


# Using Stemmer on the tokens. we  used PorterStemmer because it is not as aggressive as other stemmers and this feature helps us to keep more information. using other stemmer such as Lancaster Stemmer can lead to loss of valuable information, especially this time that we want to find adjectives in reviews

In [12]:
stm = nltk.PorterStemmer()
stem_tokes = []
for token in tokenized_reviews:
    temp=[]
    for toke in token:
        temp.append(stm.stem(toke))
    stem_tokes.append(temp)
np.array(stem_tokes)

C:\Users\smrh1\AppData\Local\Temp\ipykernel_35196\112666477.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(stem_tokes)


array([list(['strang', 'old', 'time', 'creatur', 'featur', 'mash', 'godzilla', 'vs', 'kong', 'releas', 'first', 'major', 'blockbust', 'cinema', 'sinc', 'covid', 'shutter', 'year', 'prior', 'expect', 'low', 'thank', 'rotten', 'last', 'two', 'godzilla', 'film', 'thirst', 'someth', 'anyth', 'truli', 'escapist', 'high', 'big', 'screen', 'equival', 'kid', 'smash', 'toy', 'togeth', 'becam', 'unlik', 'saviour', 'commerci', 'critic', 'three', 'year', 'later', 'normal', 'resum', 'arguabl', 'less', 'audienc', 'demand', 'anoth', 'instal', 'although', 'industri', 'could', 'definit', 'anoth', 'monster', 'hit', 'strike', 'leav', 'first', 'month', '2024', 'littl', 'weaken', 'enough', 'easili', 'market', 'simplic', 'godzilla', 'x', 'kong', 'new', 'empir', 'becom', 'swift', 'global', 'hit', 'film', 'track', 'make', '135m', 'worldwid', 'open', 'weekend', 'especi', 'shadow', 'oscar', 'win', 'godzilla', 'minu', 'one', 'predict', 'diminish', 'return', 'ventur', 'still', 'fun', 'yet', 'far', 'sloppier', 'ou

In [13]:
not_stemmed_preprocessed_news=[]
for i in tokenized_reviews:
    t=""
    for j in i:
        t=t+" "+j
    not_stemmed_preprocessed_news.append(t)
np.array(not_stemmed_preprocessed_news)

array([' strange old time creature feature mash godzilla vs kong released first major blockbuster cinemas since covid shuttered year prior expectations low thanks rotten last two godzilla films thirst something anything truly escapist high big screen equivalent kid smashing toys together became unlikely saviour commercially critically three years later normality resuming arguably less audience demand another instalment although industry could definitely another monster hit strikes leaving first months 2024 little weakened enough easily marketable simplicity godzilla x kong new empire become swift global hit film tracking make 135m worldwide opening weekend especially shadow oscar winning godzilla minus one predictably diminishing returns venture still fun yet far sloppier outing second round less win us draw start truce sorts godzilla remains king protector regular world fighting creatures week surface kong stays hollow earth magical space discovered previous film time coming end spurr

# Using Spacy model on preprocessed data that is not stemmed

In [14]:
not_stemmed=[]
nlp = spacy.load('en_core_web_sm')
for i in not_stemmed_preprocessed_news:
    document = nlp(i)
    not_stemmed.append(document)

In [15]:
sents = [sent for sent in not_stemmed[0].sents]
for x in sents:
    print([(tok.pos_,tok.lemma_) for tok in x])

[('SPACE', ' '), ('ADJ', 'strange'), ('ADJ', 'old'), ('NOUN', 'time'), ('NOUN', 'creature'), ('NOUN', 'feature'), ('NOUN', 'mash'), ('NOUN', 'godzilla'), ('ADP', 'vs'), ('PROPN', 'kong'), ('VERB', 'release'), ('ADJ', 'first'), ('ADJ', 'major'), ('NOUN', 'blockbuster'), ('NOUN', 'cinema'), ('SCONJ', 'since'), ('ADJ', 'covid'), ('ADJ', 'shuttered'), ('NOUN', 'year'), ('ADJ', 'prior'), ('NOUN', 'expectation'), ('ADJ', 'low'), ('NOUN', 'thank'), ('ADJ', 'rotten'), ('ADJ', 'last'), ('NUM', 'two'), ('NOUN', 'godzilla'), ('NOUN', 'film'), ('VERB', 'thirst'), ('PRON', 'something'), ('PRON', 'anything'), ('ADV', 'truly'), ('VERB', 'escapist'), ('ADJ', 'high'), ('ADJ', 'big'), ('NOUN', 'screen'), ('ADJ', 'equivalent'), ('NOUN', 'kid'), ('VERB', 'smash'), ('NOUN', 'toy'), ('ADV', 'together'), ('VERB', 'become'), ('ADJ', 'unlikely'), ('NOUN', 'saviour'), ('ADV', 'commercially'), ('ADV', 'critically'), ('NUM', 'three'), ('NOUN', 'year'), ('ADV', 'later'), ('NOUN', 'normality'), ('VERB', 'resume'), 

In [16]:
displacy.render(not_stemmed[0], style="dep")

# stemmed data

In [17]:
preprocessed_news=[]
for i in stem_tokes:
    t=""
    for j in i:
        t=t+" "+j
    preprocessed_news.append(t)
np.array(preprocessed_news)

array([' strang old time creatur featur mash godzilla vs kong releas first major blockbust cinema sinc covid shutter year prior expect low thank rotten last two godzilla film thirst someth anyth truli escapist high big screen equival kid smash toy togeth becam unlik saviour commerci critic three year later normal resum arguabl less audienc demand anoth instal although industri could definit anoth monster hit strike leav first month 2024 littl weaken enough easili market simplic godzilla x kong new empir becom swift global hit film track make 135m worldwid open weekend especi shadow oscar win godzilla minu one predict diminish return ventur still fun yet far sloppier outing second round less win us draw start truce sort godzilla remain king protector regular world fight creatur week surfac kong stay hollow earth magic space discov previou film time come end spur dental issu poor kong whose infect tooth mayb sad sack loneli thrust back human someth greater also play plagu dream jia kayle

# Spacy on data without any preprocess

In [18]:
nlp = spacy.load('en_core_web_sm')
doc=[]
for i in train_news:
    document = nlp(i)
    doc.append(document)

# Spacy on data preprocessed + stemmed

In [19]:
doc_preproccessed=[]
for i in preprocessed_news:
    document_preprocessed = nlp(i)
    doc_preproccessed.append(document_preprocessed)

In [20]:
sentences_a = [sent for sent in doc_preproccessed[0].sents]
for x in sentences_a:
    print([(tok.pos_,tok.lemma_) for tok in x])

[('SPACE', ' '), ('VERB', 'strang'), ('ADJ', 'old'), ('NOUN', 'time'), ('NOUN', 'creatur'), ('NOUN', 'featur'), ('NOUN', 'mash'), ('NOUN', 'godzilla'), ('ADP', 'vs'), ('PROPN', 'kong'), ('VERB', 'relea'), ('ADV', 'first'), ('ADJ', 'major'), ('PROPN', 'blockbust'), ('PROPN', 'cinema'), ('PROPN', 'sinc'), ('PROPN', 'covid'), ('PROPN', 'shutter'), ('PROPN', 'year'), ('ADV', 'prior'), ('VERB', 'expect'), ('ADJ', 'low'), ('NOUN', 'thank'), ('VERB', 'rotten'), ('ADJ', 'last'), ('NUM', 'two'), ('NOUN', 'godzilla'), ('NOUN', 'film'), ('NOUN', 'thirst'), ('NOUN', 'someth'), ('PROPN', 'anyth'), ('PROPN', 'truli'), ('PROPN', 'escapist'), ('ADJ', 'high'), ('ADJ', 'big'), ('NOUN', 'screen'), ('NOUN', 'equival'), ('NOUN', 'kid'), ('NOUN', 'smash'), ('NOUN', 'toy'), ('PROPN', 'togeth'), ('PROPN', 'becam'), ('PROPN', 'unlik'), ('NOUN', 'saviour'), ('NOUN', 'commerci'), ('NOUN', 'critic'), ('NUM', 'three'), ('NOUN', 'year'), ('ADV', 'later'), ('ADJ', 'normal'), ('NOUN', 'resum'), ('NOUN', 'arguabl'), (

In [21]:
displacy.render(doc_preproccessed[0], style="dep")

# Data with no preprocessing

In [22]:
sents = [sent for sent in doc[0].sents]
for x in sents:
    print([(tok.pos_,tok.lemma_) for tok in x])

[('PRON', 'it'), ('AUX', 'be'), ('DET', 'a'), ('ADJ', 'strange'), ('ADJ', 'old'), ('NOUN', 'time'), ('SCONJ', 'when'), ('DET', 'the'), ('NOUN', 'creature'), ('VERB', 'feature'), ('NOUN', 'mash'), ('PUNCT', '-'), ('NOUN', 'up'), ('PROPN', 'Godzilla'), ('ADP', 'vs'), ('PROPN', 'Kong'), ('AUX', 'be'), ('VERB', 'release'), ('PUNCT', ','), ('DET', 'the'), ('ADJ', 'first'), ('ADJ', 'major'), ('NOUN', 'blockbuster'), ('ADP', 'in'), ('NOUN', 'cinemas'), ('SCONJ', 'since'), ('PROPN', 'Covid'), ('VERB', 'shutter'), ('PRON', 'they'), ('DET', 'all'), ('DET', 'a'), ('NOUN', 'year'), ('ADV', 'prior'), ('PUNCT', '.')]
[('NOUN', 'expectation'), ('AUX', 'be'), ('ADJ', 'low'), ('PUNCT', ','), ('NOUN', 'thank'), ('ADP', 'to'), ('SCONJ', 'how'), ('ADJ', 'rotten'), ('DET', 'the'), ('ADJ', 'last'), ('NUM', 'two'), ('PROPN', 'Godzilla'), ('NOUN', 'film'), ('AUX', 'have'), ('AUX', 'be'), ('PUNCT', ','), ('CCONJ', 'but'), ('NOUN', 'thirst'), ('ADP', 'for'), ('PRON', 'something'), ('PUNCT', ','), ('PRON', 'anyt

In [23]:
displacy.render(doc[0], style="dep")

In [24]:
sentence_words = list(itertools.chain(*tokenized_reviews))
print ('%-16s' % 'Number of words', '%-16s' % len(sentence_words))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(sentence_words)))
avg=np.sum([len(word) for word in sentence_words])/len(sentence_words)
print ('%-16s' % 'Average word length', '%-16s' % avg)
avg_sentence_length_c=np.mean([len(' '.join(sentence)) for sentence in sentence_words])
print ('%-16s' % 'Average sentence length in characters', '%-16s' % avg_sentence_length_c)

Number of words  327502          
Number of unique words 39178           
Average word length 6.097312382825143
Average sentence length in characters 11.194624765650286


# In this cell we find out that the word "film" is the most used word in all of the reviews which is normal because our dataset is movie reviews written by theGuardian

In [25]:
mp_freqdist = FreqDist(itertools.chain(*tokenized_reviews))
top20words=mp_freqdist.most_common(40)      # show the top 20 (word, frequency) pairs
print ('%-16s' % 'word', '%-16s' % 'Frequency','%-16s' %  '% of the total')
for topword in top20words:
    percent=(topword[1]/len(tokenized_reviews))
    print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)

word             Frequency        % of the total  
film             3168             2.648829431438127
–                2969             2.4824414715719065
’                2702             2.259197324414716
”                2172             1.8160535117056855
“                2160             1.806020066889632
one              1521             1.2717391304347827
like             1453             1.2148829431438126
time             826              0.6906354515050167
us               812              0.6789297658862876
also             802              0.6705685618729097
life             776              0.6488294314381271
new              775              0.6479933110367893
movie            769              0.6429765886287625
even             706              0.5903010033444817
first            683              0.5710702341137124
director         672              0.5618729096989966
story            670              0.560200668896321
something        636              0.5317725752508361

# By using Spacy in this cell we will find all noun chunks in the reviews which contains key chunks of the sentences

In [26]:
keywords = []
for sentence in doc:
    temp=[]
    for chunk in sentence.noun_chunks:
        if chunk.text.lower() not in nlp.Defaults.stop_words:
            temp.append(chunk.text)
    keywords.append(temp)

In [27]:
np.array(keywords)

C:\Users\smrh1\AppData\Local\Temp\ipykernel_35196\2185074432.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(keywords)


array([list(['a strange old time', 'the creature', 'mash-up Godzilla', 'Kong', 'the first major blockbuster', 'cinemas', 'Covid', 'Expectations', 'the last two Godzilla films', 'thirst', 'the big screen', 'a kid', 'his toys', 'an unlikely saviour', 'normality resuming', 'arguably less audience demand', 'another instalment', 'the industry', 'another monster hit', 'the strikes', 'the first few months', 'enough easily marketable simplicity', 'Godzilla x Kong', 'a swift global hit', 'the film', 'its opening weekend', 'the shadow', 'the Oscar-winning Godzilla', 'Minus One', 'returns', 'a still fun yet far sloppier outing', 'a second round', 'a win', 'a draw', 'a truce', 'sorts', 'Godzilla', 'king', 'protector', 'the regular world', 'creatures', 'the week', 'Kong', 'the Hollow Earth', 'the previous film', 'their time', 'an end', 'some dental issues', 'poor Kong', 'whose infected tooth', 'maybe sad sack loneliness', 'humanity', 'play', 'the dreams', 'Jia', 'Kaylee Hottle', 'a bond', 'Kong', '

# Finding all of adjectives in all of the reviews

In [28]:
adjectives=[]
for i in doc:
    adjectives.append([token.lemma_ for token in i if token.pos_ == "ADJ"])

In [29]:
np.array(adjectives)

C:\Users\smrh1\AppData\Local\Temp\ipykernel_35196\586600693.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(adjectives)


array([list(['strange', 'old', 'first', 'major', 'low', 'rotten', 'last', 'escapist', 'high', 'big', 'unlikely', 'less', 'first', 'few', 'little', 'weakened', 'enough', 'marketable', 'swift', 'global', 'sloppy', 'second', 'less', 'more', 'regular', 'magical', 'other', 'previous', 'dental', 'poor', 'infected', 'sad', 'sack', 'great', 'arch', 'last', 'neat', 'dank', 'serious', 'piss', 'poor', 'more', 'refreshing', 'light', 'brash', 'many', 'pink', 'breezy', 'effective', 'human', 'less', 'perfunctory', 'dreadful', 'quippy', 'unfunny', 'luminous', 'last', 'real', 'middle', 'devoid', 'smashing', 'hard', 'basic', 'like', 'giant', 'entertaining', 'able', 'large', 'final', 'many', 'more', 'next']),
       list(['brilliant', 'exhausting', 'hybrid', 'live', '60th', 'new', 'discriminatory', 'eccentric', 'obsolete', 'indigenous', 'irish', 'cheerful', 'nice', 'popular', 'upmarket', 'artificial', 'prosperous', 'upper', 'middle', 'high', 'spirited', 'grumpy', 'pompous', 'complacent', 'tiny', 'little'

# TOP 40 most Used adjectives in all of the reviews. this was done to find out what adjectives Guardian Movie reviewer are likely to use.

In [30]:
adj_freqdist = FreqDist(itertools.chain(*adjectives))
top40words=adj_freqdist.most_common(40)      # show the top 20 (word, frequency) pairs
print ('%-16s' % 'word', '%-16s' % 'Frequency','%-16s' %  '% of the total')
for topword in top40words:
    percent=(topword[1]/len(tokenized_reviews))
    print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)

word             Frequency        % of the total  
old              613              0.5125418060200669
new              586              0.4899665551839465
good             561              0.4690635451505017
own              549              0.4590301003344482
young            544              0.45484949832775917
more             538              0.4498327759197324
other            513              0.4289297658862876
first            509              0.42558528428093645
-                491              0.4105351170568562
little           441              0.36872909698996653
big              386              0.32274247491638797
many             373              0.31187290969899667
real             371              0.3102006688963211
such             364              0.30434782608695654
great            321              0.2683946488294314
much             317              0.26505016722408026
last             316              0.26421404682274247
same             306              0.2558

# In this part we want to extract name of the movie from the headline of the review. First we load the data then we would do preprocess

In [31]:
train_headline = train["news_headline"]

In [32]:
tokenized_headlines=[]
for i, headline in enumerate(train_headline):
  temp = remove_punctuation(headline.lower().replace("’s", '').strip())
  tokenized_headlines.append(nltk.word_tokenize(temp))
# for i, headline in enumerate(tokenized_headlines):
#     tokenized_headlines[i] = [word for word in headline if word not in stopwords]

In [33]:
pd.DataFrame(tokenized_headlines)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,godzilla,x,kong,the,new,empire,review,–,breezy,forgettable,...,sequel,None,None,None,None,None,None,None,None,None
1,mary,poppins,review,–,disney,entertainment,sugar,rush,possesses,thermonuclear,...,None,None,None,None,None,None,None,None,None,None
2,kung,fu,panda,4,review,–,jack,black,and,awkwafina,...,hurricane,of,slapstick,more,miss,than,hit,None,None,None
3,ryuichi,sakamoto,opus,review,–,a,stark,emotional,finale,from,...,musician,None,None,None,None,None,None,None,None,None
4,the,persian,version,review,–,feelgood,iranian,american,comedy,with,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,yuli,the,carlos,acosta,story,review,–,unflinching,biopic,of,...,ballet,superstar,None,None,None,None,None,None,None,None
1192,missing,link,review,–,animated,bigfoot,caper,trips,up,None,...,None,None,None,None,None,None,None,None,None,None
1193,holy,lands,review,–,misjudged,jewish,pig,farmer,drama,None,...,None,None,None,None,None,None,None,None,None,None
1194,nervous,translation,review,–,child,s,eye,view,of,a,...,world,None,None,None,None,None,None,None,None,None


In [34]:
preprocessed_headlines=[]
for i in tokenized_headlines:
    t=""
    for j in i:
        t=t+" "+j
    preprocessed_headlines.append(t)
np.array(preprocessed_headlines)

array([' godzilla x kong the new empire review – breezy forgettable monster sequel',
       ' mary poppins review – disney entertainment sugar rush possesses thermonuclear brilliance',
       ' kung fu panda 4 review – jack black and awkwafina in hurricane of slapstick more miss than hit',
       ..., ' holy lands review – misjudged jewish pig farmer drama',
       ' nervous translation review – child s eye view of a perplexing world',
       ' 2040 review – an idealist s vision of a healthy earth'],
      dtype='<U117')

# Then we give sentences to model using Spacy and would extract noun chunks from it. then using regex we would extract name of the movie

In [35]:
headlines=[]
for i in preprocessed_headlines:
    document = nlp(i)
    headlines.append(document)

In [36]:
movie_nouns = []
for sentence in headlines:
    temp=[]
    for chunk in sentence.noun_chunks:
            temp.append(chunk.text)
    movie_nouns.append(temp)

In [37]:
np.array(movie_nouns)

C:\Users\smrh1\AppData\Local\Temp\ipykernel_35196\2564806642.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(movie_nouns)


array([list([' godzilla', 'x kong the new empire review', 'breezy forgettable monster sequel']),
       list([' mary poppins review', 'disney entertainment sugar rush', 'thermonuclear brilliance']),
       list([' kung fu', 'panda 4 review', 'jack black', 'awkwafina', 'hurricane', 'slapstick']),
       ..., list([' holy lands review']),
       list([' nervous translation review', 'child s eye view', 'a perplexing world']),
       list([' 2040 review', 'an idealist s vision', 'a healthy earth'])],
      dtype=object)

In [38]:
# import re
# Genres=["comedy", "drama" , "horror", "sci fi" , "documentary" , 'crime']
# movie_genres=[]
# for movie in movie_nouns:
#     for genre in Genres:
#         for item in movie:
#             match = re.search(r"^(.*?)\s*{0}".format(genre for genre in Genres), item, re.IGNORECASE)
#             if match:
#                 print(match)
#                 movie_genres.append({genre})
# movie_genres

In [39]:
movie_names=[]
for i in movie_nouns:
    for j,item in enumerate(i):
        # Use regular expression to find the movie name before the word "review"
        match = re.search(r"^(.*?)\s*review", item, re.IGNORECASE)
        if match:
            if len(i[:j])>0:
                temp=((i[:j].pop().strip())+" " + match.group(1).strip())
            else:
                temp= match.group(1).strip()
            movie_names.append([temp])
np.array(movie_names)

array([['godzilla x kong the new empire'],
       ['mary poppins'],
       ['kung fu panda 4'],
       ['ryuichi sakamoto opus'],
       ['the persian version'],
       ['the devil'],
       ['swatantra veer savarkar'],
       ['irish wish'],
       ['dolls'],
       ['phantom parrot'],
       ['the new boy'],
       ['the comedy man'],
       ['phantom parrot'],
       ['darkness'],
       ['a million days'],
       ['room the top'],
       ['frida'],
       ['copa 71'],
       ['defoe'],
       ['mum'],
       ['tropic'],
       ['spaceman'],
       ['dolls'],
       ['dune part two'],
       ['powell pressburger'],
       ['the story cymande'],
       ['eureka'],
       ['du temps time'],
       ['viktoria'],
       ['her beat'],
       ['suncoast'],
       ['nature the dry 2'],
       ['slotherhouse'],
       ['days heaven'],
       ['argylle'],
       ['migration'],
       ['american fiction'],
       ['the color purple'],
       ['dìdi'],
       ['the island'],
       ['presence'